<a href="https://colab.research.google.com/github/jatink121/FINSEARCH_Jatin/blob/main/Roadmap_3%2B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf


In [ ]:
file_path = '/content/drive/MyDrive/Finsearch/Nifty_50_Data.csv'
data = pd.read_csv(file_path)

In [ ]:
df = data.iloc[::-1]

In [ ]:
class state :
  def __init__(self, prev_val, big_mov_avg, val, mov_avg):
    self.prev_val = prev_val
    self.big_mov_avg = big_mov_avg
    self.val = val
    self.mov_avg = mov_avg
  def state_representation (self) :
    return np.array([self.prev_val , self.big_mov_avg, self.val ,self.mov_avg])
def random_action() :
  action  =  np.random.randint(0,3)
  return action
def reward(action ,present_state , Post_state) :
  if action==0  :                              #  buy
    PnL = (Post_state-present_state)
  elif action ==2:                             #sell
    PnL = (present_state - Post_state)
  else :                                   #hold
    PnL = 0
  return PnL

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, models
input_vals = tf.keras.Input(shape=(4,))
x = layers.Dense(128, activation='relu')(input_vals)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)

# Output layer with 3 neurons
output_vals = layers.Dense(3, activation='linear')(x)

# Create the model
model = models.Model(inputs=input_vals, outputs=output_vals)
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
States_list  = [np.array([0 , df['Avg2'][len(df)-1] ,df['Open'][len(df)-1] , df['Avg'][len(df)-1]])]
j=len(df)-2
while j>=0:
  temp_state = state(df['Open'][j+1], df['Avg2'][j], df['Open'][j], df['Avg'][j])
  temp_state_representation = temp_state.state_representation()
  States_list.append(temp_state_representation)
  j-=1
States = np.array(States_list)

In [ ]:
from collections import deque
import random
i=0
j=0
n_iterations = 100
n_states = len(df)
epsilon = 1.0
epsilon_decay = 0.995
min_epsilon = 0.01
discount_factor = 0.6
replay_buffer = deque(maxlen=5000)
while i< n_iterations:
  while j< n_states-1 :
    state = States[j]
    next_state = States[j+1]
    if np.random.uniform(0,1) < epsilon :
      action =  random_action ()
    else :
      action  = np.argmax(model.predict(np.expand_dims(state, axis=0)))
    PnL  = reward(action ,state[2] , next_state[2] )
    replay_buffer.append((state, action, PnL, next_state))
    j+=1
    epsilon = max(min_epsilon, epsilon * epsilon_decay)
    if j%500==0 :
      break
    if j==n_states-1 :
      j=0
      break
  batch_size =100
  if len(replay_buffer) > batch_size:
    batch = random.sample(replay_buffer, batch_size)
    states, actions, rewards, next_states = zip(*batch)
    q_next_state_list = model.predict(np.array(next_states))
    q_target_matrix = np.array(rewards) + discount_factor * np.max(q_next_state_list, axis=1)
    q_values = model.predict(np.array(states))
    q_values[np.arange(batch_size), actions] = q_target_matrix
    model.fit(np.array(states), q_values, validation_split=0.2, epochs=1)
  if i%10==0 :
    print(i,"th iteration completed")
  i+=1


In [ ]:
q_values = model.predict(np.expand_dims(States[2], axis=0))
print(q_values)

1/1 [==============================] - 0s 35ms/step
[[  1.4199518   -0.49475065 -12.586126  ]]


In [ ]:
i=2340
money_used =0
money_made =0
int_amount =0
budget=100000
profit=0
n=0
while i <2355:
  cur_state = States[i]
  next_state = States[i+1]
  action  = np.argmax(model.predict(np.expand_dims(cur_state, axis=0)))
  if int_amount>=budget and action ==0 :
    action =1
  if action ==0 :
    int_amount+=state[2]
    money_used+=state[2]
    n+=1
  elif action ==2 :
    int_amount=0
    money_made += n*cur_state[2]
    profit+= n*cur_state[2] - int_amount
    n=0
  i+=1
print((money_made + n*cur_state[2] - money_used)*100/money_used)

1/1 [==============================] - 0s 22ms/step


IndexError: ignored

In [ ]:
model.save("/content/drive/MyDrive/Finsearch/DQN_model_finsearch_final.h5")